In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_arabic_tweets.csv to cleaned_arabic_tweets.csv


In [ ]:
!pip install nlppreprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud
import string
from nlppreprocess import NLP
import pickle
# splitting data
from sklearn.model_selection import train_test_split
# Features Extraction
from sklearn.feature_extraction.text import CountVectorizer
# Features Extraction
from sklearn.feature_extraction.text import TfidfVectorizer

# Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
# from sklearn.svm import SVC

from sklearn import preprocessing

from sklearn.svm import SVC

# # Features Extraction word2vec
# from gensim.models import Word2Vec

# # Dimension reduction
# from sklearn.decomposition import PCA

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import io
df_file = pd.read_csv(io.BytesIO(uploaded['cleaned_arabic_tweets.csv']))
df_file.dropna(inplace=True)
print(len(df_file))
df_file.head()

5991


,tweet,class
0,فينو الاهبل ابن الاهبل,0
1,علي المصرييييين وجمالهم ربنا يحميهم MinaAtta,1
2,دول كتير اوي ودمهم خفيف العماره اللي انا كلها ...,1
3,انا بعد كده خلي اللي يوعني بحاجه همضي علي وصل ...,0
4,انا هنتحر,0


In [ ]:
df_file = df_file.sample(frac=1).reset_index(drop=True) # shuffel
print(len(df_file))
df_file.head()

5991


,tweet,class
0,الحياه عطياني ضهر دين امها ليه ...,0
1,شبح اضطرابات عشيه التصديق علي عشرات الاعدامات ...,1
2,المصري بيقول حزام ناسف الشروق بيقول سياره مستف...,0
3,السابع التوك شو الجمل ابو الفتوح اقرب مر...,2
4,اسمي ميشيل وانا سكرتير دعلاء وبانقله تويتات ال...,2


In [ ]:
text_list = list(df_file['tweet'])
print(len(text_list))
X = text_list
y = df_file['class']


5991


In [ ]:
def tfidf_vectorizer(df):
    '''
    Argumen:
        df dataframe of multiple reviews
    return:
        Train & test arrays that can fir to the model
    '''
# I fit the vector to all of the data
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer = tfidf_vectorizer.fit(df) 
    word_idf_weights = tfidf_vectorizer.idf_
    print("Our 10 words weights\n\n",word_idf_weights[:10])
    return tfidf_vectorizer

In [ ]:
tfidf_vectorizer = tfidf_vectorizer(text_list) 
pickle.dump(tfidf_vectorizer, open("tfidf_vectorizer_25000.pickle", "wb"))
# files.download("tfidf_vectorizer_25000.pickle")
tfidf_vectorizer = pickle.load(open("tfidf_vectorizer_25000.pickle", "rb"))

Our 10 words weights

 [8.59956824 9.00503334 9.00503334 9.00503334 9.00503334 9.00503334
 9.00503334 9.00503334 9.00503334 8.59956824]


In [ ]:
# convert y from text to label encoder and we can see all entiment labels now
LabelEncoder = preprocessing.LabelEncoder()
LabelEncoder.fit(y)
list(LabelEncoder.classes_)
y = LabelEncoder.transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print("Our training data now are: " + str(len(X_train))  + " Reviews")
print("Our testing data now are: " + str(len(X_test))  + " Reviews")
print("Our training data now are: " + str(len(y_train))  + " labels")
print("Our testing data now are: " + str(len(y_test))  + " labels")

Our training data now are: 4792 Reviews
Our testing data now are: 1199 Reviews
Our training data now are: 4792 labels
Our testing data now are: 1199 labels


In [ ]:
training_data = tfidf_vectorizer.transform(X_train)
training_data  = training_data.toarray()
testing_data = tfidf_vectorizer.transform(X_test)
testing_data  = testing_data.toarray()

In [ ]:
clf_MultinomialNB = MultinomialNB()
model = clf_MultinomialNB.fit(training_data, y_train)
predict = model.predict(training_data)
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))
predict = model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

F1 score of our training data is:  0.8401502504173621
F1 score of our testing data is:  0.5479566305254379


In [ ]:
clf_LogisticRegression = LogisticRegression(penalty='l2', solver='sag')
logistic_model = clf_LogisticRegression.fit(training_data, y_train)
predict = logistic_model.predict(training_data)
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))
predict = logistic_model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

F1 score of our training data is:  0.9311352253756261
F1 score of our testing data is:  0.5454545454545454


In [ ]:
clf_LogisticRegression = LogisticRegression(penalty='l2', solver='liblinear')
logistic_model = clf_LogisticRegression.fit(training_data, y_train)
predict = logistic_model.predict(training_data)
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))
predict = logistic_model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

F1 score of our training data is:  0.8524624373956594
F1 score of our testing data is:  0.5479566305254379
